In [284]:
import h5py
import os
import cv2
import numpy as np
import scipy.spatial.distance
import scipy.io
# import mat73
import glob
from skimage import io
from sklearn.cluster import KMeans
from numpy import moveaxis

In [285]:
File_path = '/home/cse/Desktop/Raj_M.Tech_Project/Approach4/Output_Image_Dataset/MNF/Input_For_Pixel_Replace/HS_Images_10_Output'
HS_files = sorted(os.listdir(File_path))
print("HS_files", HS_files)
save_path = '/home/cse/Desktop/Raj_M.Tech_Project/Approach4/Output_Image_Dataset/MNF/Final_Output_Pixel_Replace'

HS_files ['0001.mat']


In [286]:
# def create_dict(temp_dict, dict_size):
# # dict_size = 10
#   # temp_dict = {}
#   for i in range(dict_size):
#     temp_dict.update({i: list()})
#   return temp_dict

In [287]:
number_of_clusters = 10

In [288]:
# dict_size = 10Location_Appended_Gaussian_Results
# pixel_values = create_dict({}, dict_size)
# cluster_means = create_dict({}, dict_size)

In [289]:
# print('pixel values', pixel_values)
# print('cluster means', cluster_means)

In [290]:
def mkdir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [291]:
im_final_images_path = os.path.join(save_path, 'Location_Appended_10_Channel_Results')
mkdir(im_final_images_path)

In [292]:
def find_euclidean_distance(mean_list):
    dist = 0
    # print('len of mean_list', len(mean_list))
    # clus_dist = []
    euc_dist = []
    for i in range(len(mean_list)):
        for j in range(len(mean_list)):
            # for k in range(len(mean_list[i])):
                # print('i-j', mean_list[i][j][0])
                # print('i-k', mean_list[i][k][0])
            dist += scipy.spatial.distance.euclidean(mean_list[i], mean_list[j])
                # dist += np.linalg.norm(mean_list[i][j][0] - mean_list[i][k][0])
        euc_dist.append(dist)
        dist = 0

        # euc_dist.append(clus_dist)
        # clus_dist = []

    return euc_dist

In [293]:
# def find_euclidean_distance_using(centers):
#     euc_dist = []

#     return euc_dist

In [294]:
def build_image_using_labelling(labels, euc_dist):
    # euc_dist = np.asarray(euc_dist)
    # for label in range(euc_dist_ind, len(euc_dist)):
    for k in range(0, len(euc_dist)):
        for i in range(labels.shape[0]):
            for j in range(labels.shape[1]):
            
                # p = k
                if labels[i][j] == k:
                    labels[i][j] = euc_dist[k]
                        # print('euc_dist[label][k]', euc_dist[label][k])
                    break
    
    max_pixel_val = np.max(labels)
    labels = labels/max_pixel_val
    labels = labels * 255
    
    return labels

In [295]:
# def build_image(file_name, labels, clusters_ind, euc_dist_ind, mean_list, euc_dist):
#     for clusters in range(clusters_ind, len(mean_list), 1):
#         for label, mean in mean_list[clusters].items():
#             if np.mean(mean[0]) != 0:
#                 for i in range(labels.shape[0]):
#                     for j in range(labels.shape[1]):
#                         # print(img[i][j])
#                         if label == labels[i][j]:
#                         # print(1)
#                     #   print(euc_dist[clusters])
#                             labels[i][j] = euc_dist[clusters][euc_dist_ind]
#                 euc_dist_ind += 1
#                 # print('If euc_dist_ind', euc_dist_ind)
      
#             else:
#                 euc_dist_ind += 1
#                 # print('Else euc_dist_ind', euc_dist_ind)
#                 continue
#         euc_dist_ind = 0
  
#         # clusters += 1
#         # img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
#         # img = cv2.flip(img, 1)
    
#         max_pixel_val = np.max(labels)
#         labels = labels/max_pixel_val
#         labels = labels * 255

#         # kernel = np.ones((9, 9), np.uint8)
#         # img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

#         save_im_final_featureMap = im_final_images_path + '/' + file_name + '.png'
#         cv2.imwrite(save_im_final_featureMap, labels)

#         break
#     return

In [296]:
# def print_labels(labels):
#     for row in labels:
#         print(' '.join([str(elem) for elem in row]))

In [297]:

for file in HS_files:
  num = 0
  mean_list = []
  euc_dist_ind = 0
  clusters_ind = -1
  pixel_values = []
  cluster_means = []
# dist = 0
# clus_dist = []
  euc_dist = []
  input_img = os.path.join(File_path, file)
  # print("Reading File[%d/%d]: %s" % (num, len(HS_files), input_img))
  # print("file name", file)
  mat_dict = scipy.io.loadmat(input_img)
  for key in mat_dict:
    if type(mat_dict[key]) is np.ndarray:
    # print(type(mat_dict[key]))
    # if type(mat_dict[key]) is h5py._hl.dataset.Dataset:
      image_data = mat_dict[key]  #type: np.ndarray

  
  # print("image data shape before axis move:", image_data.shape)
  #image_data = np.swapaxes(image_data, 0, 2)
  # image_data = np.swapaxes(image_data, 0, 1)
  image_data = np.moveaxis(image_data, 0, 1)
  # image_data = np.moveaxis(image_data, 0, 1)
  # image_data = np.moveaxis(image_data, 0, 2)

  # print("image data after axis move:", image_data.shape)

  img = image_data.reshape((-1, 11))
  img = np.float32(img)
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1.0)
  ret, labels, centers = cv2.kmeans(img, number_of_clusters, None, criteria, 10, cv2.KMEANS_PP_CENTERS)

  # print('centers', centers)
  # print('labels dimension before flattening', labels.shape)
  labels = labels.flatten()
  labels = labels.reshape(768, 1024)
  # print('labels dimension after flattening', labels.shape)

  # print_labels(labels)

  file = file.rsplit('.', 1)

  for k in range(0,10):
    pixel_values = []
    cluster_mean = []
    for i in range(labels.shape[0]):
      for j in range(labels.shape[1]):
        if (labels[i][j] == k):
          pixel_values.append(image_data[i][j][:])
    cluster_mean = np.mean(pixel_values, axis = 0)

    cluster_means.append(cluster_mean)

  # Function Call for Euclidean Distance Calculation
  euc_dist = find_euclidean_distance(cluster_means)

  # euc_dist = find_euclidean_distance(centers)

  img = build_image_using_labelling(labels, euc_dist)
  # print('img', img)
  # print('img shape', img.shape)

  save_im_final_featureMap = im_final_images_path + '/' + file[0] + '.png'
  cv2.imwrite(save_im_final_featureMap, img)

print('!!!!!!!!! Image saved successfully !!!!!!!!!!')

img shape (768, 1024)
!!!!!!!!! Image saved successfully !!!!!!!!!!
